In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import pathlib
import pysam
import pyfaidx
import warnings
warnings.filterwarnings("ignore")

output_version = "dev"

# Generate metadata

In [2]:
maindir = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis"
outputdir = "/media/hieunguyen/HNSD_mini/outdir"

Vi_runs = ["R6801", "R6829Vi", "R6873", "R6914"]
Truong_runs = ["R6782", "R6823", "R6829Truong"]

regions = {
    "Vi": "Vi_amplicons.hg19.bed",
    "Truong": "Truong_amplicons.hg19.bed"
}

regions_hg38 = {
    "Vi": "Vi_amplicons.hg38.bed",
    "Truong": "Truong_amplicons.hg38.bed"
}

# mode = "directional"
mode = "non_directional"

# pic = "Truong"
pic = "Vi"

path_to_main_output = os.path.join(outputdir, "targetMethyl_analysis", output_version, f"{pic}_output", mode)
os.system(f"mkdir -p {path_to_main_output}")

print(f"All output are saved at {path_to_main_output}")


real_cpgdf = pd.read_excel(os.path.join(path_to_main_output, "00_output", f"{pic}_panel_correct_cpgdf.xlsx"))


All output are saved at /media/hieunguyen/HNSD_mini/outdir/targetMethyl_analysis/dev/Vi_output/non_directional


In [3]:
all_cov_files = [item for item in pathlib.Path(maindir).glob("target_methylation_*/06*/*.cov")]

metadata_dict = {
    "filename"  : [item.name.split(".no_deduplicated")[0] for item in all_cov_files],
    "Run" : [str(item).split("/")[6].replace("_no_dedup", "").replace("target_methylation_", "") for item in all_cov_files],
    "path": [str(item) for item in all_cov_files]
}

metadata = pd.DataFrame.from_dict(metadata_dict, orient="columns")
metadata["mode"] = metadata["Run"].apply(lambda x: "directional" if "_without_non_directional" in x else "non_directional")
metadata["Run"] = metadata["Run"].apply(lambda x: x.split("_")[0])
metadata["PIC"] = metadata["Run"].apply(lambda x: "Vi" if x in Vi_runs else "Truong")
metadata["bam_path"] = metadata["path"].apply(lambda x: str(x).replace("06_methylation_extract", "05_sorted_bam").replace(".bedGraph.gz.bismark.zero.cov", ".sorted.bam"))
metadata = metadata[(metadata["mode"] == mode) & (metadata["PIC"] == pic)]

##### regiondf for hg19
regiondf = pd.read_csv(regions[pic], sep = "\t", header = None)
regiondf.columns = ["chrom", "start", "end"]
regiondf = regiondf[["chrom", "start", "end"]]
regiondf["region_name"] = regiondf[["chrom", "start", "end"]].apply(
    lambda x: f"region_{x[0]}_{x[1]}_{x[2]}", axis = 1
)
regiondf["bam_region"] = regiondf[["chrom", "start", "end"]].apply(
    lambda x: f"{x[0].replace('chr', '')}:{x[1]}-{x[2]}", axis = 1
)

##### regiondf for hg38
regiondf_hg38 = pd.read_csv(regions_hg38[pic], sep = ",", header = None)
regiondf_hg38.columns = ["chrom", "start", "end"]
regiondf_hg38 = regiondf_hg38[["chrom", "start", "end"]]
regiondf_hg38["region_name"] = regiondf_hg38[["chrom", "start", "end"]].apply(
    lambda x: f"region_{x[0]}_{x[1]}_{x[2]}", axis = 1
)
regiondf_hg38["bam_region"] = regiondf_hg38[["chrom", "start", "end"]].apply(
    lambda x: f"{x[0].replace('chr', '')}:{x[1]}-{x[2]}", axis = 1
)

metadata

,filename,Run,path,mode,PIC,bam_path
0,TM010,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
1,TM100,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
2,TM002,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
3,TM050,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
4,TM005,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
5,TM001,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
29,3-TMH3_S7527-S7727,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
30,4-TMH4_S7528-S7728,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
31,11-TMM3_S7535-S7735,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
32,1-TMH1_S7525-S7725,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...


# Count on/off target reads

In [4]:
metadata["num_total_reads"] = metadata["bam_path"].apply(lambda x: int(pysam.samtools.view("-c", x, catch_stdout=True)))

def count_read_in_region(bam_path, region, chr_mode = False):
    all_reads = []
    bamfile = pysam.AlignmentFile(bam_path, "rb")
    if chr_mode:
        region = f"chr{region}"
    fetched_obj = bamfile.fetch(region = region)
    for read in fetched_obj:
        all_reads.append(read)
    return(len(all_reads))
if os.path.isfile(os.path.join(path_to_main_output, f"{pic}_read_count_in_region.xlsx")) == False:
    for region_name in regiondf.region_name.unique():
        bam_region = regiondf[regiondf["region_name"] == region_name]["bam_region"].values[0]
        metadata[region_name] = metadata["bam_path"].apply(lambda x: count_read_in_region(x, bam_region))
        metadata[f"pct_{region_name}"] = metadata[region_name] / metadata["num_total_reads"] * 100
        
    metadata.to_excel(os.path.join(path_to_main_output, f"{pic}_read_count_in_region.xlsx"), index = False)
else:
    print("File already exists, skip counting reads in regions. Reading existing data... ")
    metadata = pd.read_excel(os.path.join(path_to_main_output, f"{pic}_read_count_in_region.xlsx"))

File already exists, skip counting reads in regions. Reading existing data... 


# Process cov file

In [5]:
metadata.head()

,filename,Run,path,mode,PIC,bam_path,num_total_reads,region_chr16_22825587_22825729,pct_region_chr16_22825587_22825729,region_chr16_22825770_22825948,...,region_chr16_22826024_22826152,pct_region_chr16_22826024_22826152,region_chr16_22826130_22826286,pct_region_chr16_22826130_22826286,region_chr17_75369424_75369590,pct_region_chr17_75369424_75369590,region_chr17_75369739_75369887,pct_region_chr17_75369739_75369887,region_chr17_75369604_75369763,pct_region_chr17_75369604_75369763
0,TM010,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,26,0,0.000000,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,TM100,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,36,16,44.444444,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,TM002,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,32,0,0.000000,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,TM050,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,38,14,36.842105,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,TM005,R6801,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,non_directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,10,0,0.000000,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [6]:
regiondf.head()

,chrom,start,end,region_name,bam_region
0,chr16,22825587,22825729,region_chr16_22825587_22825729,16:22825587-22825729
1,chr16,22825770,22825948,region_chr16_22825770_22825948,16:22825770-22825948
2,chr16,22825920,22826053,region_chr16_22825920_22826053,16:22825920-22826053
3,chr16,22826024,22826152,region_chr16_22826024_22826152,16:22826024-22826152
4,chr16,22826130,22826286,region_chr16_22826130_22826286,16:22826130-22826286


In [7]:
all_covdf = dict()
for run in metadata.Run.unique():
      for filename in metadata[metadata["Run"] == run]["filename"].unique():
            path_to_save_cov = os.path.join(path_to_main_output, run, filename)
            os.system(f"mkdir -p {path_to_save_cov}")

            covdf = pd.read_csv(metadata[metadata["filename"] == filename]["path"].values[0], header = None, sep = "\t")
            covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
            all_covdf[filename] = covdf
            covdf["chrom"] = covdf["chrom"].apply(lambda x: str(x))
            for region in regiondf.region_name.unique():
                  chrom = str(region.split("_")[1].replace("chr", ""))
                  start = int(region.split("_")[2])
                  end = int(region.split("_")[3])
                  save_covdf = covdf[(covdf["chrom"] == chrom) & 
                        (covdf["start"] >= start) & (covdf["start"] <= end)]
                  save_covdf["CpG"] = save_covdf[["chrom", "start", "end"]].apply(lambda x: f"chr{str(x[0])}:{x[1]}-{x[2]}", axis = 1)
                  save_covdf["check_context"] = save_covdf["CpG"].apply(lambda x: "CpG_context" if x in real_cpgdf["CpG"].values else "False")
                  save_covdf.to_excel(f"{path_to_save_cov}/{region}.xlsx", index = False)
                  # print(f"Sample {filename}, Run {run}, region {region}, number of reads = {save_covdf.shape}")
                  


In [8]:
save_covdf

,chrom,start,end,meth_density,countC,countT,CpG,check_context


# Check sample: CONTROL114, R6447

## Check number of reads

In [9]:
control114_cov = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.cg_cov.bed"
control114_bam = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam"
control114_cov_hg19 = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.cg_cov.hg19.bed"


path_to_save_cov = os.path.join(path_to_main_output, "R6447", "CONTROL114")
os.system(f"mkdir -p {path_to_save_cov}")
os.system(f"mkdir -p {path_to_save_cov}/hg38")
os.system(f"mkdir -p {path_to_save_cov}/hg19")

#### data target commercial 1-based, our data is 0-based
covdf = pd.read_csv(control114_cov, header = None, sep = "\t")
covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
covdf["start"] = covdf["start"].apply(lambda x: int(x) - 1)
covdf["end"] = covdf["end"].apply(lambda x: int(x) - 1)

covdfhg19 = pd.read_csv(control114_cov_hg19, header = None, sep = "\t")
covdfhg19.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
covdfhg19["start"] = covdfhg19["start"].apply(lambda x: int(x) - 1)
covdfhg19["end"] = covdfhg19["end"].apply(lambda x: int(x) - 1)


covdfhg19["countC"] = covdf["countC"].values
covdfhg19["countT"] = covdf["countT"].values

for region in regiondf_hg38.region_name.unique():
        chrom = region.split("_")[1]
        start = int(region.split("_")[2])
        end = int(region.split("_")[3])

        covdf[(covdf["chrom"] == chrom) & 
            (covdf["start"] >= start) & (covdf["start"] <= end)].to_excel(f"{path_to_save_cov}/hg38/{region}.xlsx", index = False)
        
        print(covdf[(covdf["chrom"] == chrom) & 
            (covdf["start"] >= start) & (covdf["start"] <= end)].shape)
        
for region in regiondf.region_name.unique():
        chrom = region.split("_")[1]
        start = int(region.split("_")[2])
        end = int(region.split("_")[3])
        save_covdf = covdfhg19[(covdfhg19["chrom"] == chrom) & 
            (covdfhg19["start"] >= start) & (covdfhg19["start"] <= end)]
        save_covdf["CpG"] = save_covdf[["chrom", "start", "end"]].apply(lambda x: f"{str(x[0])}:{x[1]}-{x[2]}", axis = 1)
        save_covdf["check_context"] = save_covdf["CpG"].apply(lambda x: "CpG_context" if x in real_cpgdf["CpG"].values else "False")
                  
        save_covdf.to_excel(f"{path_to_save_cov}/hg19/{region}.xlsx", index = False)
        
        print(covdfhg19[(covdfhg19["chrom"] == chrom) & 
            (covdfhg19["start"] >= start) & (covdfhg19["start"] <= end)].shape)
        
all_covdf["CONTROL114"] = covdfhg19

(16, 6)
(19, 6)
(11, 6)
(14, 6)
(10, 6)
(12, 6)
(17, 6)
(14, 6)
(16, 6)
(19, 6)
(11, 6)
(14, 6)
(10, 6)
(12, 6)
(17, 6)
(14, 6)


In [10]:
metadata_CONTROL114 = pd.DataFrame.from_dict({
    "filename": ["CONTROL114"],
    "Run": ["R6447"],
    "bam_path": [control114_bam],
    "path": [control114_cov]
})

metadata_CONTROL114["num_total_reads"] = metadata_CONTROL114["bam_path"].apply(lambda x: int(pysam.samtools.view("-c", x, catch_stdout=True)))

for region_name in regiondf_hg38.region_name.unique():
    bam_region = regiondf_hg38[regiondf_hg38["region_name"] == region_name]["bam_region"].values[0]
    metadata_CONTROL114[region_name] = metadata_CONTROL114["bam_path"].apply(lambda x: count_read_in_region(x, bam_region, chr_mode = True))
    metadata_CONTROL114[f"pct_{region_name}"] = metadata_CONTROL114[region_name] / metadata_CONTROL114["num_total_reads"] * 100

metadata_CONTROL114.to_excel(os.path.join(path_to_main_output, f"CONTROL114_read_count_in_region.xlsx"), index = False)

[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/

## Compare methylation density between 2 samples

In [11]:
mean_methyldf = pd.DataFrame(data = regiondf.region_name.unique(), columns = ["region"])

def get_mean_methyl_in_region(region, filename, remove_non_cpg = True):

    if filename == "CONTROL114":
        df = pd.read_excel(f"{os.path.join(path_to_main_output, 'R6447', 'CONTROL114')}/hg19/{region}.xlsx")
        run = "R6447"
    else:
        run = metadata[metadata["filename"] == filename]["Run"].values[0]
        df = pd.read_excel(f"{os.path.join(path_to_main_output, run, filename)}/{region}.xlsx")
    if df.shape[0] == 0:
        return "no data available"
    else:
        if remove_non_cpg:
            # keep only CpG context Cs
            df = df[df["check_context"] == "CpG_context"]
        mean_methyl = df.meth_density.mean()
        return mean_methyl

for filename in all_covdf.keys():
    mean_methyldf[filename] = mean_methyldf["region"].apply(lambda x: get_mean_methyl_in_region(x, filename))

mean_methyldf.to_excel(os.path.join(path_to_main_output, f"{pic}_mean_methyl_in_region_compare_CONTROL114.xlsx"), index = False)


In [12]:
merge_double_checkdf = pd.DataFrame(data = regiondf.region_name.unique(), columns = ["region"])
for run in metadata.Run.unique():
      for filename in metadata[metadata["Run"] == run]["filename"].unique():
            path_to_save_double_check_file = os.path.join(path_to_main_output, run, filename, "check_MethylDensity_and_Count")
            os.system(f"mkdir -p {path_to_save_double_check_file}")
            check_region_countdf = metadata[metadata["filename"] == filename]\
                [[item for item in metadata.columns if "region" in item and "pct" not in item]]\
                    .T.reset_index()
            check_region_countdf.columns = ["region", f"count_{filename}"]
            check_methyldf = mean_methyldf[["region", filename]]
            double_checkdf = check_methyldf.merge(check_region_countdf, right_on = "region", left_on = "region")
            double_checkdf.to_excel(f"{path_to_save_double_check_file}/check_methylDensity_and_Count_{filename}.xlsx", index = False)
            merge_double_checkdf = merge_double_checkdf.merge(double_checkdf, right_on = "region", left_on = "region")
merge_double_checkdf.to_excel(os.path.join(path_to_main_output, f"check_MethylDensity_and_Count_all_Samples.xlsx"))